In [1]:
import pandas as pd
import re

import folium
import requests

In [2]:
Congestion = pd.read_csv("../data/Seoul_Congestion_week_2022.csv", encoding='cp949', index_col=0)
Elevator = pd.read_csv("../data/elevator_area.csv", index_col=0)
Elevator = Elevator.rename(columns = {"면적" : "역면적"})

Transfer = pd.read_csv("../data/서울시_환승역.csv", encoding='cp949')

def preprocess_station_name(station_name):
    station_name = re.sub(r'\([^)]*\)', '', station_name)
    
    if not station_name.endswith('역'):
        station_name += '역'
    
    return station_name

Transfer['환승역명'] = Transfer['환승역명'].apply(preprocess_station_name)

Transfer.환승역명 = ["이수역" if i == "총신대입구역" else i for i in Transfer.환승역명 ]

Transfer = list(set(Transfer["환승역명"]))

idx = [i for i in range(len(Congestion)) if Congestion.iloc[i].역명 in Transfer]
Transfer_df = Congestion.iloc[idx]

In [3]:
# 총 환승역과 혼잡도 데이터의 환승역 차이
set(Transfer) - set(Transfer_df.역명)

{'부평구청역', '선정릉역'}

In [4]:
Lease = pd.read_csv('../data/Lease.csv', encoding='euc-kr') 
Lease.columns = ['연번', '상가유형', '호선', '역명', '상가번호', '면적', '영업업종', '계약시작일자','계약종료일자', '월임대료', '사업진행단계']
Lease['역명'] = Lease['역명'].apply(preprocess_station_name)
Lease.역명 = ["이수역" if i == "총신대입구역" else i for i in Lease.역명 ]
Lease = Lease[Lease['상가유형']=='공실']

In [5]:
Lease = pd.concat([Lease.groupby("역명")["면적"].sum(), Lease.groupby("역명").count()["상가유형"]], axis = 1)
Lease = Lease.rename(columns = {"상가유형" : "공실수", "면적" : "공실면적"})
Lease = Lease.reset_index()

In [6]:
Transfer_name = pd.read_csv("../data/서울시_환승역.csv", encoding='cp949')[["호선", "환승역명"]]
Transfer_name['환승역명'] = Transfer_name['환승역명'].apply(preprocess_station_name)
Transfer_name.환승역명 = ["이수역" if i == "총신대입구역" else i for i in Transfer_name.환승역명 ]

In [7]:
Transfer_name["자치구"] = 0
for i in range(len(Transfer_name)):
    line, subway = Transfer_name[["호선", "환승역명"]].iloc[i].values
    url = "https://dapi.kakao.com/v2/local/search/keyword.json?query=" + subway + str(line) + "호선"
    headers = {"Authorization" : "KakaoAK 3206f186fc6467db13a17a82c6e3e668"}
    region = requests.get(url, headers = headers).json()["documents"][0]["address_name"].split()[1]
    Transfer_name.자치구[i] = region


C:\Users\Ryzen\AppData\Local\Temp\ipykernel_18388\553557616.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Transfer_name.자치구[i] = region


In [8]:
Transfer_name = Transfer_name[["환승역명","자치구"]]
Transfer_name = Transfer_name.drop_duplicates()
Transfer_name.loc[126, "자치구"] = "구로구"
Transfer_name.loc[57, "자치구"] = "중구"
Transfer_name = Transfer_name.drop_duplicates()
Transfer_name = Transfer_name.dropna()


In [9]:
Population = pd.read_csv('../data/Population.csv', header=2, usecols=['동별(1)','동별(2)','소계','소계.1', '인구밀도 (명/㎢)'])
Population.columns = ['합계', '자치구' ,'세대', '인구', '인구밀도']

In [10]:
Population = Population[["자치구","세대","인구","인구밀도"]]

In [11]:
Population = pd.merge(Transfer_name, Population, how = "inner",on = "자치구")

In [12]:
Population = Population.rename(columns = {"환승역명" : "역명"})

In [13]:
Population['역명'] = Population['역명'].apply(preprocess_station_name)
Population.역명 = ["이수역" if i == "총신대입구역" else i for i in Population.역명 ]

In [14]:
parcel_2308 = pd.read_excel('../data/택배/2308서울도착.xlsx')[["수하인_구명","택배총량"]]
parcel_2307 = pd.read_excel('../data/택배/2307서울도착.xlsx')[["수하인_구명","택배총량"]]
parcel_2306 = pd.read_excel('../data/택배/2306서울도착.xlsx')[["수하인_구명","택배총량"]]
parcel_2305 = pd.read_excel('../data/택배/2305서울도착.xlsx')[["수하인_구명","택배총량"]]
parcel_2304 = pd.read_excel('../data/택배/2304서울도착.xlsx')[["수하인_구명","택배총량"]]
parcel_2303 = pd.read_excel('../data/택배/2303서울도착.xlsx')[["수하인_구명","택배총량"]]
parcel_2302 = pd.read_excel('../data/택배/2302서울도착.xlsx')[["수하인_구명","택배총량"]]
parcel_2301 = pd.read_excel('../data/택배/2301서울도착.xlsx')[["수하인_구명","택배총량"]]
parcel_2212 = pd.read_excel('../data/택배/2212서울도착.xlsx')[["수하인_구명","택배총량"]]
parcel_2211 = pd.read_excel('../data/택배/2211서울도착.xlsx')[["수하인_구명","택배총량"]]
parcel_2210 = pd.read_excel('../data/택배/2210서울도착.xlsx')[["수하인_구명","택배총량"]]
parcel_2209 = pd.read_excel('../data/택배/2209서울도착.xlsx')[["수하인_구명","택배총량"]]

In [15]:
idx = [i for i in range(len(parcel_2308)) if parcel_2308.iloc[i].수하인_구명 in list(set(Transfer_name.자치구))]
parcel_2308 = parcel_2308.iloc[idx]
idx = [i for i in range(len(parcel_2307)) if parcel_2307.iloc[i].수하인_구명 in list(set(Transfer_name.자치구))]
parcel_2307 = parcel_2307.iloc[idx]
idx = [i for i in range(len(parcel_2306)) if parcel_2306.iloc[i].수하인_구명 in list(set(Transfer_name.자치구))]
parcel_2306 = parcel_2306.iloc[idx]
idx = [i for i in range(len(parcel_2305)) if parcel_2305.iloc[i].수하인_구명 in list(set(Transfer_name.자치구))]
parcel_2305 = parcel_2305.iloc[idx]
idx = [i for i in range(len(parcel_2304)) if parcel_2304.iloc[i].수하인_구명 in list(set(Transfer_name.자치구))]
parcel_2304 = parcel_2304.iloc[idx]
idx = [i for i in range(len(parcel_2303)) if parcel_2303.iloc[i].수하인_구명 in list(set(Transfer_name.자치구))]
parcel_2303 = parcel_2303.iloc[idx]
idx = [i for i in range(len(parcel_2302)) if parcel_2302.iloc[i].수하인_구명 in list(set(Transfer_name.자치구))]
parcel_2302 = parcel_2302.iloc[idx]
idx = [i for i in range(len(parcel_2301)) if parcel_2301.iloc[i].수하인_구명 in list(set(Transfer_name.자치구))]
parcel_2301 = parcel_2301.iloc[idx]
idx = [i for i in range(len(parcel_2212)) if parcel_2212.iloc[i].수하인_구명 in list(set(Transfer_name.자치구))]
parcel_2212 = parcel_2212.iloc[idx]
idx = [i for i in range(len(parcel_2211)) if parcel_2211.iloc[i].수하인_구명 in list(set(Transfer_name.자치구))]
parcel_2211 = parcel_2211.iloc[idx]
idx = [i for i in range(len(parcel_2210)) if parcel_2210.iloc[i].수하인_구명 in list(set(Transfer_name.자치구))]
parcel_2210 = parcel_2210.iloc[idx]
idx = [i for i in range(len(parcel_2209)) if parcel_2209.iloc[i].수하인_구명 in list(set(Transfer_name.자치구))]
parcel_2209 = parcel_2209.iloc[idx]

In [16]:
parcel_2308 = parcel_2308.rename(columns = {"택배총량" : "택배_2308"})
parcel_2307 = parcel_2307.rename(columns = {"택배총량" : "택배_2307"})
parcel_2306 = parcel_2306.rename(columns = {"택배총량" : "택배_2306"})
parcel_2305 = parcel_2305.rename(columns = {"택배총량" : "택배_2305"})
parcel_2304 = parcel_2304.rename(columns = {"택배총량" : "택배_2304"})
parcel_2303 = parcel_2303.rename(columns = {"택배총량" : "택배_2303"})
parcel_2302 = parcel_2302.rename(columns = {"택배총량" : "택배_2302"})
parcel_2301 = parcel_2301.rename(columns = {"택배총량" : "택배_2301"})
parcel_2212 = parcel_2212.rename(columns = {"택배총량" : "택배_2212"})
parcel_2211 = parcel_2211.rename(columns = {"택배총량" : "택배_2211"})
parcel_2210 = parcel_2210.rename(columns = {"택배총량" : "택배_2210"})
parcel_2209 = parcel_2209.rename(columns = {"택배총량" : "택배_2209"})

In [17]:
parcel_2308 = parcel_2308.groupby("수하인_구명").mean().reset_index()
parcel_2307 = parcel_2307.groupby("수하인_구명").mean().reset_index()
parcel_2306 = parcel_2306.groupby("수하인_구명").mean().reset_index()
parcel_2305 = parcel_2305.groupby("수하인_구명").mean().reset_index()
parcel_2304 = parcel_2304.groupby("수하인_구명").mean().reset_index()
parcel_2303 = parcel_2303.groupby("수하인_구명").mean().reset_index()
parcel_2302 = parcel_2302.groupby("수하인_구명").mean().reset_index()
parcel_2301 = parcel_2301.groupby("수하인_구명").mean().reset_index()
parcel_2212 = parcel_2212.groupby("수하인_구명").mean().reset_index()
parcel_2211 = parcel_2211.groupby("수하인_구명").mean().reset_index()
parcel_2210 = parcel_2210.groupby("수하인_구명").mean().reset_index()
parcel_2209 = parcel_2209.groupby("수하인_구명").mean().reset_index()


In [18]:
Parcel = pd.merge(parcel_2308, parcel_2307, how = "inner", on = "수하인_구명")
Parcel = pd.merge(Parcel, parcel_2306, how = "inner", on = "수하인_구명")
Parcel = pd.merge(Parcel, parcel_2305, how = "inner", on = "수하인_구명")
Parcel = pd.merge(Parcel, parcel_2304, how = "inner", on = "수하인_구명")
Parcel = pd.merge(Parcel, parcel_2303, how = "inner", on = "수하인_구명")
Parcel = pd.merge(Parcel, parcel_2302, how = "inner", on = "수하인_구명")
Parcel = pd.merge(Parcel, parcel_2301, how = "inner", on = "수하인_구명")
Parcel = pd.merge(Parcel, parcel_2212, how = "inner", on = "수하인_구명")
Parcel = pd.merge(Parcel, parcel_2211, how = "inner", on = "수하인_구명")
Parcel = pd.merge(Parcel, parcel_2210, how = "inner", on = "수하인_구명")
Parcel = pd.merge(Parcel, parcel_2209, how = "inner", on = "수하인_구명")


In [19]:
Parcel = Parcel.rename(columns = {"수하인_구명":"자치구"})

In [20]:
Total_df = pd.merge(Transfer_df, Elevator, how = "inner", on = "역명")
Total_df = pd.merge(Total_df, Population, how = "inner", on = "역명")
Total_df = pd.merge(Total_df, Lease, how = "inner", on = "역명")
Total_df = pd.merge(Total_df, Parcel, how = "inner", on = "자치구")
Total_df.index = Total_df["역명"]
Total_df = Total_df.loc[:,~Total_df.columns.isin(["역명"])] 
Total_df = Total_df.loc[:,~Total_df.columns.isin(["자치구"])]
Total_df = Total_df.rename(columns = {"역내개수" : "역내엘리베이터개수",
                           "역내용량최대" : "역내엘리베이터용량최대",
                           "역외개수" : "역외엘리베이터개수",
                           "역외용량최대" : "역외엘리베이터용량최대",
                           "총개수" : "엘리베이터총개수"})

In [22]:
Total_df.to_csv("../data/Total_data_62col.csv")

In [23]:
time5_9 = Total_df[["5시30분", "6시00분", "6시30분", "7시00분", "7시30분", "8시00분", "8시30분"]]
time5_9 = pd.DataFrame(time5_9.max(axis='columns'))
time5_9.columns = ["time5_9"]

time9_12 = Total_df[["9시00분", "9시30분", "10시00분", "10시30분", "11시00분", "11시30분"]]
time9_12 = pd.DataFrame(time9_12.max(axis='columns'))
time9_12.columns = ["time9_12"]

time12_15 = Total_df[["12시00분", "12시30분", "13시00분", "13시30분", "14시00분", "14시30분"]]
time12_15 = pd.DataFrame(time12_15.max(axis='columns'))
time12_15.columns = ["time12_15"]

time15_18 = Total_df[["15시00분", "15시30분", "16시00분", "16시30분", "17시00분", "17시30분"]]
time15_18 = pd.DataFrame(time15_18.max(axis='columns'))
time15_18.columns = ["time15_18"]

time18_21 = Total_df[["18시00분", "18시30분", "19시00분", "19시30분", "20시00분", "20시30분"]]
time18_21 = pd.DataFrame(time18_21.max(axis='columns'))
time18_21.columns = ["time18_21"]

time21_24 = Total_df[["21시00분", "21시30분", "22시00분", "22시30분", "23시00분", "23시30분", "00시00분", "00시30분"]]
time21_24 = pd.DataFrame(time21_24.max(axis='columns'))
time21_24.columns = ["time21_24"]
Total_df_2 = pd.concat([time5_9, time9_12, time12_15, time15_18, time18_21, time21_24, 
                      Total_df[['역면적', '역내엘리베이터개수', '역내엘리베이터용량최대','역외엘리베이터개수', '역외엘리베이터용량최대', 
                                '엘리베이터총개수', '세대', '인구', '인구밀도', '공실면적', '공실수', '택배_2308', '택배_2307', '택배_2306', 
                                '택배_2305', '택배_2304', '택배_2303', '택배_2302', '택배_2301', '택배_2212', '택배_2211', '택배_2210', '택배_2209']]], axis = 1)

In [24]:
Total_df_2.to_csv("../data/Total_data_29col.csv")

In [25]:
Total_df_2

,time5_9,time9_12,time12_15,time15_18,time18_21,time21_24,역면적,역내엘리베이터개수,역내엘리베이터용량최대,역외엘리베이터개수,...,택배_2306,택배_2305,택배_2304,택배_2303,택배_2302,택배_2301,택배_2212,택배_2211,택배_2210,택배_2209
역명,,,,,,,,,,,,,,,,,,,,,
가락시장역,99.20,77.10,44.00,91.00,114.65,48.55,19754.13,4.0,1000.0,2,...,3616.163065,3689.159681,3293.369352,3650.398077,3446.153684,3895.641975,3858.453510,3955.503922,3671.992188,3903.434322
공덕역,85.30,39.75,32.95,64.85,72.05,32.15,15953.14,0.0,0.0,0,...,2395.696252,2454.881764,2198.772549,2464.071154,2318.789030,2542.516461,2574.626186,2660.887574,2426.418468,2584.031983
합정역,92.60,46.00,37.30,55.10,74.75,54.50,22750.80,4.0,1000.0,4,...,2395.696252,2454.881764,2198.772549,2464.071154,2318.789030,2542.516461,2574.626186,2660.887574,2426.418468,2584.031983
군자역,99.45,76.65,38.90,76.60,91.50,50.55,19806.00,0.0,0.0,0,...,1858.085657,1886.422846,1690.322134,1872.435701,1744.656118,1920.508230,1966.655238,2047.261811,1878.889980,2000.025586
노원역,37.30,29.45,18.30,27.85,34.25,21.35,17160.94,4.0,1000.0,4,...,2203.926733,2281.038306,1996.617647,2239.599229,2119.388186,2356.039256,2410.402277,2467.522682,2284.129412,2418.497881
태릉입구역,59.80,47.15,29.00,52.35,62.95,34.35,26250.28,0.0,0.0,0,...,2203.926733,2281.038306,1996.617647,2239.599229,2119.388186,2356.039256,2410.402277,2467.522682,2284.129412,2418.497881
당산역,96.60,50.30,38.85,60.05,85.20,54.05,6385.00,4.0,750.0,2,...,2287.600791,2328.770916,2137.001965,2340.536538,2252.234177,2487.571721,2476.423150,2514.923529,2335.116371,2488.703390
대림역,85.85,58.75,39.35,68.85,80.50,44.10,15835.53,4.0,1000.0,4,...,2043.021782,2069.956088,1863.319527,2046.430769,1929.578059,2169.948240,2201.363118,2248.816568,2073.602362,2148.989429
동대문역,94.20,62.65,48.80,79.85,89.70,49.70,15245.30,6.0,1000.0,4,...,1088.467066,1112.062500,1010.399209,1121.254826,1022.329114,1149.565762,1173.772901,1204.092885,1113.892644,1155.738854
